<h2> ====================================================</h2>
 <h1>MA477 - Theory and Applications of Data Science</h1> 
  <h1>Lesson 25: Neural Networks (Part 2)</h1> 
 
 <h4>Dr. Valmir Bucaj</h4>
 <br>
 United States Military Academy, West Point, AY20-2
<h2>=====================================================</h2>

<h2>Lecture Outline</h2>

<ul>
    <li> Review of the LogReg Algorithm(viewed as a Perceptron)</li>
    <li>Importance of Vectorization?</li>
    <li> Vectorization of LogReg Algorithm
    <ol>
        <li> Forward Propagation</li>
        <li> Backward Propagation</li>
        <li>Vectorized Algorithm</li>
        </ol></li>
    <li>Python Implementation</li>
   
        
        
   </li>
    
 </ul>

<h2>Algorithm</h2>

For this specific example, one iteration/step of the gradient descent the algorithm would be as follows:

Initiate $w=(w_1,w_2), b$ and $J=0,dw=0,db=0$


For $i$ in $range(1,m)$:

$\hspace{1cm}z^{(i)}=w^Tx^{(i)}+b^{(i)}$

$\hspace{1cm}a^{(i)}=\sigma(z^{(i)})$

$\hspace{1cm}J+=-\left[y^{(i)}\log\left(a^{(i)}\right)+\left(1-y^{(i)}\right)\log\left(1-a^{(i)}\right)\right]$

$\hspace{1cm}$--- This is where Forward Propagation ends and Back Propagation begins! ---

$\hspace{1cm}dz^{(i)}=a^{(i)}-y^{(i)}$

$\hspace{1cm}db+=dz^{(i)}$


$\hspace{1cm}$ For j in range(1,$n_x=2$): 

$\hspace{2cm}dw_j+=x_j^{(i)}\,dz^{(i)}$



$J:=J/m,\, dw_j:=dw_j/m,\, db:=db/m$

(Updating Weights:)

$w_j:=w_j-\alpha\,dw_j$

$b :=b-\alpha\, db$



<h2>Importance of Vectorization</h2>

The purpose of vectorization for us will be to get rid of the <b> for loops</b>. For loops slow down computations significantly, as we will demonstrate below with a simple example.



In [1]:
import numpy as np
import time

In [21]:
x=np.random.randn(10000000)
y=np.random.randn(10000000)

<h4>Non-Vectorized Computation</h4>

In [29]:
start=time.time()

tot=0
for i,j in zip(x,y):
    tot+=i*j
end=time.time()

print('Non-Vectorized Product: {}; \n Time: {}'.format(tot,1000*(end-start))+' in ms')

Non-Vectorized Product: 1865.2496132198976; 
 Time: 2779.8073291778564 in ms


<h4>Vectorized Computation</h4>

In [30]:
start=time.time()

tot=np.dot(x,y)

end=time.time()

t=end-start

print('Vectorized Product: {} \n Time: {}'.format(tot, 1000*t)+ ' in ms')

Vectorized Product: 1865.2496132195715 
 Time: 9.82975959777832 in ms


As we can see, the non-vectorized version took over 300 times longer to run. This was a single for-loop in a very simple scenario. We can certainly imagine more complicated computations taking place inside for-loops (e.g. in a NN), which would slow down the computations significantly. That's why, whenever possible it is imperative that we vectorize the code. We shall do so for LogReg below.

<h2>Vectorizing LogReg </h2>

The first thing we are going to do is get rid of the second loop in our algorithm; that is, the loop that individually updates all of the weight changes $dw_j$. We can vectorize this step by defining 

$$dw=\begin{bmatrix}dw_1\\ \vdots \\ dw_{n_x}\end{bmatrix}$$

Then, we begin by initializing the weight changes as follows:

$$dw=\begin{bmatrix}0\\ \vdots\\0\end{bmatrix}_{n_x\times 1}$$

Then we can get rid of the following <b>for-loop</b>: 

$\hspace{1cm}$ For j in range(1,$n_x$): 

$\hspace{2cm}dw_j+=x_j^{(i)}\,dz^{(i)}$

by simply rewriting it as $$dw+=x^{(i)}dz^{(i)}=\begin{bmatrix}x_1^{(i)}dz^{(i)}\\ \vdots \\x_m^{(i)}dz^{(i)}\end{bmatrix}\hspace{.4cm} \text{ where } x^{(i)}=\begin{bmatrix}x_1^{(i)}\\ \vdots \\x_m^{(i)}\end{bmatrix}$$

So, for each training sample $i$ all of the weight changes get updated simultaneously.

In what follows we get rid of the outside for-loop by vectorizing it as well.



<h4>Forward Propagation</h4>

Suppose we have $m$ training samples $\left\{x^{(i)}\right\}_{i=1}^m$ where $x^{(i)}\in \mathbb{R}^{n_x}$. Let's repackage the training samples into an $n_x$ by $m$ matrix $X$, by inserting each training sample $x^{(i)}$ as a column of $X$:


$$X=\Big[x^{(1)}\dots x^{(m)}\Big]\, \text{ where } \, x^{(i)}=\begin{bmatrix}x_1^{(i)}\\ \vdots \\ x_{n_x}^{(i)}\end{bmatrix},\, \text{ so } X\in \mathbb{R}^{n_x\times m}$$

Similarly, let the weight and bias vector be $$ w=\begin{bmatrix}w_1\\\vdots\\ w_{n_x}\end{bmatrix},\hspace{.5cm} B=\big[b^{(1)}\dots b^{(m)}\big]$$

Then, we can compute the forward propogation of the NN as follows:

\begin{align*}
Z&=w^TX+B\\
&=\Big[w^Tx^{(1)}+b^{(1)}\dots w^Tx^{(m)}+b^{(m)}\Big]\\
&=\Big[z^{(1)}\dots z^{(m)}\Big]_{1\times m}
\end{align*}

Then,

\begin{align*}
A&=\sigma(Z)\\
&=\Big[\sigma\left(z^{(1)}\right)\dots \sigma\left(z^{(m)}\right)\Big]\\
&=\Big[a^{(1)}\dots a^{(m)}\Big]_{1\times m}
\end{align*}

So, we are done with the forward propagation; that is, with computing the output of the NN without ever using a single for-loop!

<h4>Backward Propagation</h4>

Recall, that in <b>backpropagation</b> the key piece was to compute $dz$ since it is the only additional info we need to compute $dw$ and $db$. So, in the vectorized version we may define:

$$dZ=\Big[dz^{(1)}\dots dz^{(m)}\Big]_{1\times m}, \hspace{.4cm} \text{ where } dz^{(i)}=a^{(i)}-y^{(i)}.$$

So, if we also define $$Y=\big[y^{(1)}\dots y^{(m)}\big]_{1\times m}$$ then we get 

$$dZ=A-Y$$


So, we are finally in a position to compute $db$ and $dw$.

$$db=\frac{1}{m}\sum_{i=1}^mdz^{(i)}\Rightarrow \fbox{$db=\frac{1}{m}np.sum(dZ)$}$$

and

$$dw=\frac{1}{m}XdZ^T=
\begin{bmatrix}
x_1^{(1)}& x_1^{(2)}&\dots &x_1^{(m)}\\
x_2^{(1)}&x_2^{(2)}&\dots &x_2^{(m)}\\
\vdots &&&\vdots\\
x_{n_x}^{(1)}&x_{n_x}^{(2)}&\dots &x_{n_x}^{(m)}
\end{bmatrix}
\begin{bmatrix}
dz^{(1)}\\
dz^{(2)}\\
\vdots\\
dz^{(m)}
\end{bmatrix}
=
\begin{bmatrix}\frac{1}{m}\sum_{i=1}^{m}x_1^{(i)}dz^{(i)}\\ \vdots\\ \frac{1}{m}\sum_{i=1}^{m}x_{n_x}^{(i)}dz^{(i)}\end{bmatrix}_{n_x\times 1}$$

We are done! Now we can implement the vectorized version of the LogReg Algorithm!

<h4>Vectorized Algorithm</h4>

We can complete one pass (forward and backward propagation) withoug using a single <b>for-loop</b>. However, if we want to perform more than one iteration (which in practice we always do), then we still need to have one loop. Below we descrbe the vectorized version of the LogReg Algorithm viewed as a NN (This is the same as a Perceptron!):

For <b>iter</b> in range(1,N):

$\hspace{1cm}Z=w^TX+B$

$\hspace{1cm}A=\sigma(Z)$

$\hspace{1cm}dZ=A-Y$

$\hspace{1cm}db=\frac{1}{m}np.sum(dZ)$

$\hspace{1cm}dw=\frac{1}{m}XdZ^T$

$\hspace{1cm}J+=-\frac{1}{N}\big[Y\log(A^T)+(1-Y)\log(1-A^T)\big]$

$\hspace{1cm} w:=w-\alpha dw$

$\hspace{1cm} b:=b-\alpha db$

<h2>Python Implementation</h2>

<font size='5' color='red'>Exercise:</font> <font size=4>Implement the above algorithm (Perceptron) in Python. Once you've done so, then train the Perceptron to make predictions using the airline satisfaction data set. </font>

In [957]:
#Sigmoid function

def sigmoid(z):
    return 1/(1+np.exp(-z))

In [958]:
def weight_bias_init(X):
    #W is a X.shape[0] by 1 matrix (each row of X represents a feature)
    #b is a 1 by X.shape[0]
    
    W=np.random.normal(scale=0.01,size=(X.shape[0],1))
    b=0
    #np.zeros(shape=(1,X.shape[1]))
    #print(W.T.shape)
    return W,b

In [959]:
def forward_propagation(X,W,b):

    y_out=np.matmul(W.T,X)+b
    
    y_out=sigmoid(y_out)
    #print(y_out[0])
    return y_out[0]
    

In [960]:
def cost_funct(X,y_in,y_out):
    m=X.shape[1]
    return -(1/m)*(np.matmul(y_in,np.log(y_out.T))+np.matmul(1-y_in,np.log(1-y_out.T)))
    

In [961]:
def weight_update(X,W,b,y_in,y_out,learning_rate):
    m=X.shape[1]
    
    dZ=y_out-y_in
    dw=(1/m)*np.matmul(X,dZ.T)
    db=(1/m)*np.sum(dZ)
    
    W=W-learning_rate*dw
    b=b-db
    
    return W,b
    
    

In [962]:
def train_model(X,y,n_iter,learning_rate):
    """
    X=input data
    y=response variable
    n_iter=number of iterations to run
    learning_rate=rate at which to update the weights
    
    """
    #weight and bias initialization
    W,b=weight_bias_init(X)
    
    #forward pass
    
    for i in range(1,n_iter):
        
        cost=0
        
        y_out=forward_propagation(X,W,b)
        
        cost+=cost_funct(X,y_in=y,y_out=y_out)/n_iter
        print('Iteration {} ... Cost: {:.4f}'.format(i,cost))
        
        W,b=weight_update(X,W,b,y,y_out,learning_rate)
        
    print('Total Cost {}'.format(cost))
    
    return W,b
    
    

In [963]:
def model_predict(X,W,b):
    y_pred=[]
    y_out=forward_propagation(X,W,b)
    #print(y_out)
    
    for y in y_out:
        if y>0.5:
            y_pred.append(1)
        else:
            y_pred.append(0)
    return y_out, y_pred
        

In [964]:
def index_marks(n_cols, batch_size):
    return range(batch_size, math.ceil(n_cols / batch_size) * batch_size, batch_size)

def split_dataframe(data_frame, batch_size):
    indices = index_marks(data_frame.shape[1], batch_size)
    data=np.split(data_frame, indices,axis=1)
#     X=data.drop(data.columns[-1],axis=1)
#     y=data[data.columns[-1]]
    return data

In [965]:
#This is Optional!

def train_model_batch(X,y,learning_rate,epochs,batch_size):
    """
    X=input data
    y=response variable
    n_iter=number of iterations to run
    learning_rate=rate at which to update the weights
    
    """
    #weight and bias initialization
    W,b=weight_bias_init(X)
    
    X.loc['target',X.columns]=y
#     print(X.head())
   

    total_batches=int(X.shape[1]/batch_size)
    
    #forward pass
    
    
    for epoch in range(epochs):
        X_lst=split_dataframe(X,batch_size)
        cost=0
        
        for i in range(total_batches):
            y_new=X_lst[i].loc[X_lst[i].index[-1]]
            X_new=X_lst[i].drop(X_lst[i].index[-1])
            
            X_new=np.array(X_new)
            y_new=np.array(y_new)
            
               
            y_out=forward_propagation(X_new,W,b)

            W,b=weight_update(X_new,W,b,y_new,y_out,learning_rate)
        
        y_o=forward_propagation(np.array(X.drop(X.index[-1])),W,b)
       
        cost+=cost_funct(X,y_in=np.array(y),y_out=y_o)/total_batches
        print('Epoch {} ... Cost: {:.4f}'.format(epoch,cost))
        
    print('Total Cost {}'.format(cost))
    
    return W,b

<h4>We import the airline satisfaction data</h4>

In [966]:
import pandas as pd

In [980]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.metrics import accuracy_score, recall_score
import random

In [993]:
airline=pd.read_csv('airline_train.csv',index_col=[0])
airline.drop(airline.columns[:2],inplace=True,axis=1)
airline.dropna(inplace=True)
airline.reset_index(inplace=True,drop=True)



rows=random.sample(list(airline.index),35000)
airline=airline.loc[rows]

airline.reset_index(inplace=True,drop=True)
airline['satisfaction']=airline['satisfaction'].apply(lambda x: 1 if x=='satisfied' else 0)
X=airline.drop('satisfaction',axis=1)
y=airline['satisfaction']


#Convert qualitative variables to numerical dummy variables

cols=['Gender','Customer Type', 'Type of Travel','Class']

X=pd.get_dummies(data=X,columns=cols,drop_first=True)


In [994]:
X.head()

Age  Flight Distance  Inflight wifi service  \
0   44              964                      2   
1   22             2153                      3   
2   18              262                      5   
3   47              634                      3   
4   58             3585                      3   

   Departure/Arrival time convenient  Ease of Online booking  Gate location  \
0                                  3                       3              3   
1                                  4                       3              1   
2                                  4                       5              4   
3                                  1                       1              1   
4                                  3                       3              3   

   Food and drink  Online boarding  Seat comfort  Inflight entertainment  ...  \
0               5                3             4                     2.0  ...   
1               5                3             4                     5.0  ...   
2               4                5             4                     4.0  ...   
3               2                3             4                     3.0  ...   
4               1                4             4                     3.0  ...   

   Checkin service  Inflight service  Cleanliness  Departure Delay in Minutes  \
0                1                 2            3                         0.0   
1                5                 5            5                        10.0   
2                5                 1            4                         0.0   
3                4                 3            4                         0.0   
4                4                 3            4                         0.0   

   Arrival Delay in Minutes  Gender_Male  Customer Type_disloyal Customer  \
0                       0.0            1                                0   
1                       5.0            0                                0   
2                       0.0            1                                1   
3                       0.0            0                                0   
4                       0.0            1                                0   

   Type of Travel_Personal Travel  Class_Eco  Class_Eco Plus  
0                               0          0               0  
1                               1          1               0  
2                               0          1               0  
3                               0          1               0  
4                               0          0               0  

[5 rows x 23 columns]

In [995]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=11)

In [996]:
scaler=StandardScaler()

scaled_train=scaler.fit_transform(X_train)
X_train_sc=pd.DataFrame(scaled_train,columns=X_train.columns,index=X_train.index)


scaled_val=scaler.fit_transform(X_val)
X_val_sc=pd.DataFrame(scaled_val,columns=X_val.columns,index=X_val.index)


In [997]:
#Because we are using NumPy under the hood, we need to convert dataframes to numpy arrays first

X_t=np.array(X_train_sc.T)
y_t=np.array(y_train)
X_v=np.array(X_val_sc.T)
y_v=np.array(y_val)

In [998]:
#Training the model

W,b=train_model(X_t,y_t,learning_rate=0.015,n_iter=1000)

Iteration 1 ... Cost: 0.0007
Iteration 2 ... Cost: 0.0007
Iteration 3 ... Cost: 0.0007
Iteration 4 ... Cost: 0.0007
Iteration 5 ... Cost: 0.0007
Iteration 6 ... Cost: 0.0007
Iteration 7 ... Cost: 0.0007
Iteration 8 ... Cost: 0.0007
Iteration 9 ... Cost: 0.0007
Iteration 10 ... Cost: 0.0007
Iteration 11 ... Cost: 0.0007
Iteration 12 ... Cost: 0.0007
Iteration 13 ... Cost: 0.0007
Iteration 14 ... Cost: 0.0007
Iteration 15 ... Cost: 0.0007
Iteration 16 ... Cost: 0.0007
Iteration 17 ... Cost: 0.0007
Iteration 18 ... Cost: 0.0007
Iteration 19 ... Cost: 0.0007
Iteration 20 ... Cost: 0.0007
Iteration 21 ... Cost: 0.0007
Iteration 22 ... Cost: 0.0007
Iteration 23 ... Cost: 0.0007
Iteration 24 ... Cost: 0.0007
Iteration 25 ... Cost: 0.0007
Iteration 26 ... Cost: 0.0007
Iteration 27 ... Cost: 0.0007
Iteration 28 ... Cost: 0.0007
Iteration 29 ... Cost: 0.0007
Iteration 30 ... Cost: 0.0007
Iteration 31 ... Cost: 0.0007
Iteration 32 ... Cost: 0.0007
Iteration 33 ... Cost: 0.0007
Iteration 34 ... Co

Iteration 268 ... Cost: 0.0006
Iteration 269 ... Cost: 0.0006
Iteration 270 ... Cost: 0.0006
Iteration 271 ... Cost: 0.0006
Iteration 272 ... Cost: 0.0006
Iteration 273 ... Cost: 0.0006
Iteration 274 ... Cost: 0.0006
Iteration 275 ... Cost: 0.0006
Iteration 276 ... Cost: 0.0006
Iteration 277 ... Cost: 0.0006
Iteration 278 ... Cost: 0.0006
Iteration 279 ... Cost: 0.0006
Iteration 280 ... Cost: 0.0006
Iteration 281 ... Cost: 0.0006
Iteration 282 ... Cost: 0.0006
Iteration 283 ... Cost: 0.0006
Iteration 284 ... Cost: 0.0006
Iteration 285 ... Cost: 0.0006
Iteration 286 ... Cost: 0.0006
Iteration 287 ... Cost: 0.0006
Iteration 288 ... Cost: 0.0006
Iteration 289 ... Cost: 0.0006
Iteration 290 ... Cost: 0.0006
Iteration 291 ... Cost: 0.0006
Iteration 292 ... Cost: 0.0006
Iteration 293 ... Cost: 0.0006
Iteration 294 ... Cost: 0.0006
Iteration 295 ... Cost: 0.0006
Iteration 296 ... Cost: 0.0006
Iteration 297 ... Cost: 0.0006
Iteration 298 ... Cost: 0.0006
Iteration 299 ... Cost: 0.0006
Iteratio

Iteration 536 ... Cost: 0.0007
Iteration 537 ... Cost: 0.0007
Iteration 538 ... Cost: 0.0007
Iteration 539 ... Cost: 0.0007
Iteration 540 ... Cost: 0.0007
Iteration 541 ... Cost: 0.0007
Iteration 542 ... Cost: 0.0007
Iteration 543 ... Cost: 0.0007
Iteration 544 ... Cost: 0.0007
Iteration 545 ... Cost: 0.0007
Iteration 546 ... Cost: 0.0007
Iteration 547 ... Cost: 0.0007
Iteration 548 ... Cost: 0.0007
Iteration 549 ... Cost: 0.0007
Iteration 550 ... Cost: 0.0007
Iteration 551 ... Cost: 0.0007
Iteration 552 ... Cost: 0.0007
Iteration 553 ... Cost: 0.0007
Iteration 554 ... Cost: 0.0007
Iteration 555 ... Cost: 0.0007
Iteration 556 ... Cost: 0.0007
Iteration 557 ... Cost: 0.0007
Iteration 558 ... Cost: 0.0007
Iteration 559 ... Cost: 0.0007
Iteration 560 ... Cost: 0.0007
Iteration 561 ... Cost: 0.0007
Iteration 562 ... Cost: 0.0007
Iteration 563 ... Cost: 0.0007
Iteration 564 ... Cost: 0.0007
Iteration 565 ... Cost: 0.0007
Iteration 566 ... Cost: 0.0007
Iteration 567 ... Cost: 0.0007
Iteratio

Iteration 804 ... Cost: 0.0007
Iteration 805 ... Cost: 0.0007
Iteration 806 ... Cost: 0.0007
Iteration 807 ... Cost: 0.0007
Iteration 808 ... Cost: 0.0007
Iteration 809 ... Cost: 0.0007
Iteration 810 ... Cost: 0.0007
Iteration 811 ... Cost: 0.0007
Iteration 812 ... Cost: 0.0007
Iteration 813 ... Cost: 0.0007
Iteration 814 ... Cost: 0.0007
Iteration 815 ... Cost: 0.0007
Iteration 816 ... Cost: 0.0007
Iteration 817 ... Cost: 0.0007
Iteration 818 ... Cost: 0.0007
Iteration 819 ... Cost: 0.0007
Iteration 820 ... Cost: 0.0007
Iteration 821 ... Cost: 0.0007
Iteration 822 ... Cost: 0.0007
Iteration 823 ... Cost: 0.0007
Iteration 824 ... Cost: 0.0007
Iteration 825 ... Cost: 0.0007
Iteration 826 ... Cost: 0.0007
Iteration 827 ... Cost: 0.0007
Iteration 828 ... Cost: 0.0007
Iteration 829 ... Cost: 0.0007
Iteration 830 ... Cost: 0.0007
Iteration 831 ... Cost: 0.0007
Iteration 832 ... Cost: 0.0007
Iteration 833 ... Cost: 0.0007
Iteration 834 ... Cost: 0.0007
Iteration 835 ... Cost: 0.0007
Iteratio

In [999]:
y_prob,y_pred=model_predict(X_v,W,b)

In [1000]:
accuracy_score(y_v,y_pred)

0.6732857142857143

<h4>Batch Training</h4>

Typically, instead of feeding in the entire data set into the gradient descent algorithm, you do it in batches. 

In [1037]:
W,b=train_model_batch(X_train_sc.T,y_train,learning_rate=0.01,epochs=15,batch_size=500)

Epoch 0 ... Cost: 0.0114
Epoch 1 ... Cost: 0.0111
Epoch 2 ... Cost: 0.0109
Epoch 3 ... Cost: 0.0108
Epoch 4 ... Cost: 0.0108
Epoch 5 ... Cost: 0.0109
Epoch 6 ... Cost: 0.0110
Epoch 7 ... Cost: 0.0111
Epoch 8 ... Cost: 0.0112
Epoch 9 ... Cost: 0.0113
Epoch 10 ... Cost: 0.0114
Epoch 11 ... Cost: 0.0115
Epoch 12 ... Cost: 0.0116
Epoch 13 ... Cost: 0.0117
Epoch 14 ... Cost: 0.0118
Total Cost 0.011836245516275561


In [1038]:
X_v=np.array(X_val_sc.T)
y_v=np.array(y_val)

In [1039]:
y_out,y_pred=model_predict(X_v,W,b)

In [1040]:
accuracy_score(y_v,y_pred)

0.6788571428571428